# 第1章: 基本的なプロンプト構造

- [レッスン](#レッスン)
- [演習](#演習)
- [サンプルプレイグラウンド](#サンプルプレイグラウンド)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

AnthropicはレガシーのClaude APIと現在のMessages APIの2つのAPIを提供しています。このチュートリアルでは、Messages APIのみを使用します。

最小限のMessages APIを使用したClaudeへの呼び出しには、以下のパラメータが必要です：
- `model`: 呼び出すモデルの[APIモデル名](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`: 生成を停止する前の最大トークン数。Claudeはこの最大値に達する前に停止する場合があります。このパラメータは生成する絶対的な最大トークン数のみを指定します。さらに、これはハードストップであり、Claudeが単語や文の途中で生成を停止する可能性があります。

- `messages`: 入力メッセージの配列。私たちのモデルは、交互に`user`と`assistant`の会話ターンで動作するように訓練されています。新しい`Message`を作成する際、messagesパラメータで以前の会話ターンを指定し、モデルは会話の次の`Message`を生成します。
  - 各入力メッセージは`role`と`content`を持つオブジェクトでなければなりません。単一の`user`ロールメッセージを指定するか、複数の`user`と`assistant`メッセージを含めることができます（その場合、交互でなければなりません）。最初のメッセージは常にユーザーの`role`を使用する必要があります。

また、以下のようなオプションのパラメータもあります：
- `system`: システムプロンプト - これについては後で詳しく説明します。
  
- `temperature`: Claudeの応答の変動性の度合い。これらのレッスンと演習では、`temperature`を0に設定しています。

すべてのAPIパラメータの完全なリストについては、[APIドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html)をご覧ください。

### 例

Claudeが正しくフォーマットされたプロンプトにどのように応答するか見てみましょう。以下の各セルについて、セルを実行（`shift+enter`）すると、Claudeの応答がブロックの下に表示されます。

In [ ]:
# Prompt
PROMPT = "こんにちわ、Claude。お元気ですか?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "あなたは海の色をなんと呼んでいますか?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "セリーヌ・ディオンは何年生まれですか?"

# Print Claude's response
print(get_completion(PROMPT))

次に、正しいMessages APIのフォーマットを含まないプロンプトを見てみましょう。これらの不正なフォーマットのプロンプトに対して、Messages APIはエラーを返します。

まず、`messages`配列に`role`と`content`フィールドがないMessages API呼び出しの例を示します。

> ⚠️ **警告:** プロンプトのmessagesパラメータの不正なフォーマットにより、以下のセルはエラーを返します。これは予期された動作です。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"こんにちは、Claude。お元気ですか?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

ここでは、`user`と`assistant`の役割を交互に使用していないプロンプトを示します。

> ⚠️ **警告:** `user`と`assistant`の役割が交互になっていないため、Claudeはエラーメッセージを返します。これは予期された動作です。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "セリーヌ・ディオンは何年生まれですか?"},
          {"role": "user", "content": "また、彼女について他に知っていることはありますか?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

`user`と`assistant`のメッセージは**必ず交互**でなければならず、メッセージは**必ず`user`のターンから始まる**必要があります。プロンプトに複数の`user`と`assistant`のペアを含めることができます（多ターンの会話をシミュレートするかのように）。また、Claudeが途中から続けられるように、最後の`assistant`メッセージに言葉を入れることもできます（これについては後の章で詳しく説明します）。

#### システムプロンプト

**システムプロンプト**も使用できます。システムプロンプトは、「ユーザー」のターンで質問やタスクを提示する前に、**Claudeにコンテキスト、指示、ガイドラインを提供する**方法です。

構造的に、システムプロンプトは`user`と`assistant`のメッセージのリストとは別に存在し、したがって別の`system`パラメータに属します（ノートブックの[セットアップ](#セットアップ)セクションの`get_completion`ヘルパー関数の構造を見てください）。

このチュートリアルでは、システムプロンプトを使用する可能性がある場合はどこでも、completions関数に`system`フィールドを提供しています。システムプロンプトを使用したくない場合は、単に`SYSTEM_PROMPT`変数を空の文字列に設定してください。

#### システムプロンプトの例

In [ ]:
# System prompt
SYSTEM_PROMPT = "あなたの回答は常に、会話を深める一連の批判的思考の質問であるべきです（自分の質問に答えを提供しないでください）。実際にユーザーの質問に答えないでください。"

# Prompt
PROMPT = "なぜ空は青いのですか?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

なぜシステムプロンプトを使用するのでしょうか？**適切に書かれたシステムプロンプトは、Claudeのパフォーマンスを様々な方法で向上させる**ことができます。例えば、Claudeがルールや指示に従う能力を向上させることができます。詳細については、Claudeでの[システムプロンプトの使用方法](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)に関するドキュメントをご覧ください。

ここで、いくつかの演習に進みます。レッスンのプロンプトを上記のコンテンツを変更せずに試してみたい場合は、レッスンノートブックの一番下までスクロールして[**サンプルプレイグラウンド**](#サンプルプレイグラウンド)をご覧ください。

---

## 演習
- [演習 1.1 - 3までの数え方](#演習-11---3までの数え方)
- [演習 1.2 - システムプロンプト](#演習-12---システムプロンプト)

### 演習 1.1 - 3までの数え方
適切な`user` / `assistant`フォーマットを使用して、以下の`PROMPT`を編集し、Claudeに**3まで数えさせてください**。出力には、あなたの解答が正しいかどうかも表示されます。

In [ ]:
# Prompt - これは変更すべき唯一のフィールドです
PROMPT = "[このテキストを置き換えてください]"

# Get Claude's response
response = get_completion(PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Claudeの応答と対応する評価を表示
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_1_hint)

### 演習 1.2 - システムプロンプト

`SYSTEM_PROMPT`を修正して、Claudeが3歳の子供のように応答するようにしてください。

In [ ]:
# システムプロンプト - これは変更すべき唯一のフィールドです
SYSTEM_PROMPT = "[このテキストを置き換えてください]"

# プロンプト
PROMPT = "なぜ空は青いのですか?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# 演習の正解を評価する関数
def grade_exercise(text):
    return bool(re.search(r"わぁ", text) or re.search(r"だから", text))

# Claudeの応答と対応する評価を表示
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解かれました：", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_2_hint)

### おめでとうございます！

ここまですべての演習を解いたなら、次の章に進む準備ができています。プロンプトエンジニアリングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試したり、プロンプトを微調整してClaudeの応答にどのような影響があるかを確認したりできる領域です。

In [ ]:
# プロンプト
PROMPT = "こんにちは、Claude。お元気ですか?"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "あなたは海の色をなんと呼んでいますか?"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "セリーヌ・ディオンは何年生まれですか?"

# Claudeの応答を表示
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"こんにちは、Claude。お元気ですか?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Claudeの応答を表示
print(response[0].text)

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "セリーヌ・ディオンは何年生まれですか?"},
          {"role": "user", "content": "また、彼女について他に知っていることはありますか?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Claudeの応答を表示
print(response[0].text)

In [ ]:
# システムプロンプト
SYSTEM_PROMPT = "あなたの回答は常に、会話を深める一連の批判的思考の質問であるべきです（自分の質問に答えを提供しないでください）。実際にユーザーの質問に答えないでください。"

# プロンプト
PROMPT = "なぜ空は青いのですか?"

# Claudeの応答を表示
print(get_completion(PROMPT, SYSTEM_PROMPT))